# Chapter 2 OpenAI Function Calling OpenAI Function Calling

This chapter mainly introduces the new features provided by OpenAI's API. In recent months, OpenAI has fine-tuned some new models, namely `gpt-3.5-turbo-0613` and `gpt-4-0613`.

If we have a tool function that needs to be called in some specific situations, after fine-tuning, these models can pass in new parameters, and these new parameters can be used to automatically determine whether to call the tool function. If it is determined that the tool function needs to be called, the tool function and the corresponding input parameters will be returned.

- [1. New parameters of OpenAI function](#1 new parameters of openai function)
- [1.1 Simple example: get the current weather](#11-Simple example to get the current weather)
- [1.2 New parameters: functions](#12-New parameters functions)
- [1.3 Related prompt call results](#13-Relevant prompt call results)
- [1.4 Unrelated prompt call results](#14-Unrelated prompt call results)
- [2. Function Call parameter mode](#2 function-call parameter mode)
- [2.1 Automatically determine whether to call](#21-Automatically determine whether to call) 
- [2.2 Forced not to call](#22-Forced not to call) 
- [2.3 Forced to call](#23-Forced to call) 
- [III. Function call and function execution](#III Function call and function execution) 
- [IV. English version prompt](#IV English version prompt)

## 1. New parameters of OpenAI functions

First, we directly define and use `OPENAI_API_KEY` to facilitate subsequent calls to OpenAI's API interface and use its functions.

In [1]:
import os
import openai

os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"
openai.api_key = os.environ['OPENAI_API_KEY']

### 1.1 Simple example: get the current weather

First, we use the first example of OpenAI and define a function called `get_current_weather`. Normally, getting the current weather is something that the language model itself cannot fully do. Therefore, we hope to be able to combine the language model with such a function to enhance it with current information.

In this function, we fix the returned value, such as the temperature is fixed to 22 degrees Celsius, but in actual applications, this may involve calling a weather API or some external knowledge source.

In [28]:
import json

# Define a function to get the current weather for a given location
def get_current_weather(location, unit="摄氏度"):
"""Get the current weather at the specified location"""
# Example function that simulates and returns the same weather conditions
# In a real application environment, this could be a weather API
# Create a dictionary of weather information
    weather_info = {
        "location": location,  # 天气的位置
        "temperature": "22",  # 温度
        "unit": unit,  # 温度单位，默认为摄氏度
        "forecast": ["晴", "多云"],  # 天气预报
    }
# Convert weather information to a string in JSON format and return it
    return json.dumps(weather_info)

### 1.2 New parameter: functions

So how do we pass such functions to the language model? OpenAI introduced a new parameter called `functions`, through which we can pass a list of function definitions. Since we only have one function, there is only one element in the list. This is a JSON object with several different parameters.

- name: the name of the function
- description: the description of the function
- parameters: the parameter object, which has some property settings
- type: type
- properties: It is an object itself, and the description of the corresponding parameters is passed in. In the example we can see that there are two elements, `location` and `unit`. Each of these elements has a type, which is a string, and then a description. `unit` is an external parameter setting, for example, here we want it to be Celsius or Fahrenheit, so we can define its type and enumerated parameter values ​​here.
- required: required parameters, for example, the parameter we need here is `location`.

In the function definition, `description` and the parameters in `properties` are very important because these will be passed directly to the language model, and the language model will use these descriptions to decide whether to use this function.

In [29]:
# Define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "获取指定位置的当前天气情况",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "城市和省份，例如：北京，北京市",
                },
                "unit": {"type": "string", "enum": ["摄氏度", "华氏度"]},
            },
            "required": ["location"],
        },
    }
]

### 1.3 Related prompts and call results

Next, we can define a question about the weather, such as "What's the weather like in Beijing?", and then use OpenAI's function to call the conversation API. First, we have to select a newer model, such as `gpt-3.5-turbo-0613`, and then we pass in the function defined above to see the final response result.

From the results, the role of the returned message is the assistant, the content is empty, but there is a function call parameter `function_call`, which contains two objects, `name` and `arguments`. `Name` is `get_current_weather`, which is the same name as the function we passed, and then `arguments` is this JSON format dictionary, which contains the parameters we need.

In [ ]:
import openai

# Define input message
messages = [
    {
        "role": "user",
        "content": "北京的天气怎么样?"
    }
]

# Call OpenAI's ChatCompletion API to get a response
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions # 传入function参数
)

# Print the response result
print(response)

{
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"location\": \"\u5317\u4eac\uff0c\u5317\u4eac\u5e02\",\n  \"unit\": \"\u6444\u6c0f\u5ea6\"\n}",
          "name": "get_current_weather"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1709602992,
  "id": "chatcmpl-8zE7kIlpxzjsiuxH4q6wtRh8CLDH3",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 30,
    "prompt_tokens": 95,
    "total_tokens": 125
  }
}


We can also see the response parameters, for example, here there are two parameters, namely `location` and `unit`.

In [ ]:
# Print the incoming parameters
print(response["choices"][0]["message"]["function_call"]["arguments"])

{
  "location": "北京，北京市",
  "unit": "摄氏度"
}


If we look closely at the response, we will find that the content is now empty, `function_call` is a dictionary, and the `arguments` parameter in `function_call` is also a JSON dictionary. Therefore, we can use `json.loads` to load it into a Python dictionary. The parameters it returns can be passed directly to the `get_current_weather` function we defined above.

We will find that using OpenAI to make function calls does not directly call the tool function. We still need to call the tool function. It just tells us which function to call, that is, the name, and what the parameters of the function should be. And because it does not execute the function, if we encounter some problems when using `json.loads` to decode, it is actually a problem with the model, so this part can consider taking some protective measures when writing tool functions, and this point will be discussed later.

In [ ]:
# Get the message information in the response
response_message = response["choices"][0]["message"]

print(response_message)

{
  "content": null,
  "function_call": {
    "arguments": "{\n  \"location\": \"\u5317\u4eac\uff0c\u5317\u4eac\u5e02\",\n  \"unit\": \"\u6444\u6c0f\u5ea6\"\n}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}


In [ ]:
# Print parameters
print(response["choices"][0]["message"]["function_call"]["arguments"])

{
  "location": "北京，北京市",
  "unit": "摄氏度"
}


In [ ]:
# Convert a JSON string to a Python object
args = json.loads(response_message["function_call"]["arguments"])

# Call the get_current_weather function and pass in the parameter args
get_current_weather(args)

'{"location": {"location": "\\u5317\\u4eac\\uff0c\\u5317\\u4eac\\u5e02", "unit": "\\u6444\\u6c0f\\u5ea6"}, "temperature": "22", "unit": "\\u6444\\u6c0f\\u5ea6", "forecast": ["\\u6674", "\\u591a\\u4e91"]}'

### 1.4 Unrelated prompt call results

Next, let's discuss what will happen if the question asked is unrelated to the function, that is, unrelated to the weather, and what kind of information will be returned? From the results, we can see that the content returned is normal and there is no `function_call` parameter, that is, the language model determines that the tool function is not used.

In [ ]:
# Not related to weather reminder call
messages = [
    {
        "role": "user",
        "content": "你好!",
    }
]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)

print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "\u4f60\u597d\uff01\u6709\u4ec0\u4e48\u6211\u53ef\u4ee5\u5e2e\u52a9\u4f60\u7684\u5417\uff1f",
        "role": "assistant"
      }
    }
  ],
  "created": 1709602993,
  "id": "chatcmpl-8zE7lZ3keV5JGVTFqvAWOGxit4cXj",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 88,
    "total_tokens": 107
  }
}


Due to encoding issues, we can print the corresponding returned text information separately

In [ ]:
print(response["choices"][0]["message"]["content"])

你好！有什么我可以帮助你的吗？


## 2. Function Call parameter mode

There are 3 modes for the `function_call` parameter. We can pass other parameters `function_call` to force the model to use or not use a function.

1. By default, it is set to `auto`, which means the model chooses by itself.

2. In the second mode, we can force it to call a function. If we want to always return a function

3. Another mode is `none`. This forces the language model not to use any function provided.

### 2.1 Automatically determine whether to call

The `auto` mode is that the large model chooses whether to return parameters by itself. This part is also the default. All the above methods are `auto` modes

### 2.2 Force not to call

The `none` mode is not important for this example, because the content `hello` does not require a function call, so we see that it is not used.

In [9]:
# Irrelevant prompts are forced not to be called
messages = [
    {
        "role": "user",
        "content": "你好",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none", # 传入参数强制不调用
)
print(response)
print(response["choices"][0]["message"]["content"])

{
  "id": "chatcmpl-8nfuv5tlp6UaSwUfmeHppJkQyFpCW",
  "object": "chat.completion",
  "created": 1706849893,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u4f60\u597d\uff01\u6709\u4ec0\u4e48\u53ef\u4ee5\u5e2e\u52a9\u4f60\u7684\u5417\uff1f"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 88,
    "completion_tokens": 17,
    "total_tokens": 105
  },
  "system_fingerprint": null
}
你好！有什么可以帮助你的吗？


What will happen if we force not to call the tool function when we need it (that is, use the second mode)? From the result, it still has normal `role` and `content`, but because we force not to call the function, it tries to return the correct parameters, but it is not correct.

In [10]:
# Related prompts are forced not to be called
messages = [
    {
        "role": "user",
        "content": "北京天气怎么样?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none", # 传入参数强制不调用
)
print(response)
print(response["choices"][0]["message"]["content"])

{
  "id": "chatcmpl-8nfv0wFX6tqcaoU2gT4KqICymXDxa",
  "object": "chat.completion",
  "created": 1706849898,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u8bf7\u7a0d\u7b49\uff0c\u6211\u4e3a\u60a8\u67e5\u627e\u5317\u4eac\u7684\u5929\u6c14\u60c5\u51b5\u3002"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 95,
    "completion_tokens": 18,
    "total_tokens": 113
  },
  "system_fingerprint": null
}
请稍等，我为您查找北京的天气情况。


### 2.3 Forced Calls

The last pattern of the `function_call` parameter is to force a function call, and the method is also very simple. Just pass the name of the function in the parameter. In this part, we specify `name` as `get_current_weather`, which will force it to use the `get_current_weather` function. If we look at the results, we can actually see that this `function_call` object is returned, and there are some parameters with `name` as `get_current_weather`.

Just for fun, it passes a parameter that has nothing to do with weather, and then forces it to use the `get_current_weather` function, but there is absolutely no information in the parameters we pass in about how it should call the function. So here, it made up the parameters of Beijing, Beijing, and if we run it again, it will keep calling the parameters of Beijing.

We can also try to use different model functions to call, different parameter values, different input messages, but there is a thing to note here. First of all, the function itself and the description will count towards the token usage limit passed to OpenAI. So if we run this, we can see that the prompt token returned is 95. If we comment out `functions` and `function_call`, we can see the promptThe token count is reduced to 15. Because the OpenAI model has a limit on tokens, when constructing a message to pass to OpenAI, you now need to pay attention not only to the length of the message, but also the length of the function you pass.

In [13]:
# No need to prompt for mandatory function call
messages = [
    {
        "role": "user",
        "content": "你好!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}, # 强制调用函数get_current_weather
)
print(response)
print(response["choices"][0]["message"]["function_call"]["arguments"])

{
  "id": "chatcmpl-8nfvkQDayf7kmYNdx4G84dkl1L5YC",
  "object": "chat.completion",
  "created": 1706849944,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"\u5317\u4eac\uff0c\u5317\u4eac\u5e02\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 95,
    "completion_tokens": 12,
    "total_tokens": 107
  },
  "system_fingerprint": null
}
{
  "location": "北京，北京市"
}


## 3. Function calls and execution functions

Finally, let's look at how to pass these function calls and the results of actually executing the function calls back to the language model. This is important because usually we want to use the language model to determine the function to call, then run the function, but then pass it back to the language model to get the final response.

We will go through this process, first we ask a question to get a response with the `function call` parameter, and then we add this message to our message list. Then we can simulate calling the `get_current_weather` function with the parameters provided by the language model and save it to a variable `observation`. Then we define a new message list to represent the result of the function just called. An important point here is that `role` is equal to `function`, which tells the language model that this is the response of calling the function. In addition, the name of the function `name` and the `content` variable are passed, set to the `observation` calculated above.

If we then use this message list to call the language model, we can see that the language model answers very well: the weather in Beijing is currently 22 degrees Celsius, and the weather is mainly sunny, but there are also cloudy conditions.

In [ ]:
# Not related to weather reminder call
messages = [
    {
        "role": "user",
        "content": "你好!",
    }
]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)

print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "\u4f60\u597d\uff01\u6709\u4ec0\u4e48\u6211\u53ef\u4ee5\u5e2e\u52a9\u4f60\u7684\u5417\uff1f",
        "role": "assistant"
      }
    }
  ],
  "created": 1709602993,
  "id": "chatcmpl-8zE7lZ3keV5JGVTFqvAWOGxit4cXj",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 88,
    "total_tokens": 107
  }
}


Due to encoding issues, we can only accept the corresponding returned text information

In [ ]:
print(response["choices"][0]["message"]["content"])

你好！有什么我可以帮助你的吗？


## 4. English version tips

**1.1 Simple example: Get the current weather**

In [17]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
"""Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

**1.2 Define function**

In [18]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

**1.3 Related prompt call results**

In [19]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

print(response)

{
  "id": "chatcmpl-8nfyFSmPyo4RoV79D5Urk1QzLCf1C",
  "object": "chat.completion",
  "created": 1706850099,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  },
  "system_fingerprint": null
}


**1.4 Irrelevant prompt call results**

In [20]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

**2.1 Automatically determine whether to call**

Automatically determine whether to call a function and whether the prompt is related to the function

In [21]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

{
  "id": "chatcmpl-8nfyHfC32MVE2gpE6HZxtmC7f7Mvi",
  "object": "chat.completion",
  "created": 1706850101,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


**2.2 Forced not to call**

Irrelevant prompts Forced not to call, the result is normal

In [22]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-8nfyfrOoacEQCPO3O2sYLukAdbxX4",
  "object": "chat.completion",
  "created": 1706850125,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 77,
    "completion_tokens": 9,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


**2.3 Forced call**

Forced call irrelevant to the problem

In [24]:

messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8nfyzFFeUyzJDYVbdiVDgPEtNrnKE",
  "object": "chat.completion",
  "created": 1706850145,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 12,
    "total_tokens": 95
  },
  "system_fingerprint": null
}


Forces the problem call to work, the result is the same as `auto`

In [26]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8nfzav6WbYIDLPHXYwaBXTM0byxaj",
  "object": "chat.completion",
  "created": 1706850182,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 89,
    "completion_tokens": 11,
    "total_tokens": 100
  },
  "system_fingerprint": null
}
